In [2]:
import pandas as pd
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset


/home/hydra/Desktop/genAi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('./data.csv')
df.head()

,query_id,customer_query,ideal_response
0,1,"Hi, I want to know the status of my order #12345.","To check your order status, please visit your ..."
1,2,What is your return policy for sale items?,Sale items can be returned for store credit wi...
2,3,Do you have this dress in blue? Product ID: XY...,Let me check that for you. (After a pause) It ...
3,4,How do I care for a silk blouse?,"For silk blouses, we recommend gentle hand was..."
4,5,I received the wrong size for my shoes. Order ...,I'm so sorry to hear that you received the wro...


In [5]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

# 1. Prepare sample data (questions + answers) - Load from CSV
try:
    df = pd.read_csv("data.csv")
    # Rename columns to match "input_text" and "target_text" expected by the preprocess_function
    df.rename(columns={'customer_query': 'input_text', 'ideal_response': 'target_text'}, inplace=True)
    # Prepend "question: " to input_text as in your original sample
    df['input_text'] = "question: " + df['input_text'].astype(str)
    data = df.to_dict(orient='records')
    dataset = Dataset.from_list(data)
    print(f"Successfully loaded {len(data)} rows from data.csv")
except FileNotFoundError:
    print("Error: data.csv not found. Please make sure the file is in the same directory.")
    exit() # Exit if the file isn't found

# 2. Load tokenizer and model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 3. Tokenize inputs and targets
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
    # Tokenize targets (labels)
    labels = tokenizer(text_target=targets, max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 4. Training arguments without evaluation_strategy to avoid version error
training_args = TrainingArguments(
    output_dir="./t5-finetuned-fashion",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_steps=10,
    push_to_hub=False,
    no_cuda=False,  # Set True if no GPU available
)

# 5. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 6. Start fine-tuning
trainer.train()

# 7. Save fine-tuned model and tokenizer
trainer.save_model("./t5-finetuned-fashion")
tokenizer.save_pretrained("./t5-finetuned-fashion")

print("Fine-tuning complete and model saved.")

Successfully loaded 600 rows from data.csv


Map: 100%|███████████████████████████| 600/600 [00:00<00:00, 9670.79 examples/s]
/home/hydra/Desktop/genAi/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,8.076600
20,5.142800
30,2.902000
40,2.489000
50,2.563500
60,2.128300
70,1.990100
80,2.188400
90,1.918600
100,1.963400


Fine-tuning complete and model saved.


In [6]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# 1. Load the fine-tuned model and tokenizer
model_path = "./t5-finetuned-fashion"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# 2. Define a function to generate responses
def get_model_response(question):
    # Prepend "question: " to the input, just like during training
    input_text = "question: " + question
    
    # Tokenize the input question
    inputs = tokenizer(input_text, return_tensors="pt", max_length=64, truncation=True, padding="max_length")
    
    # Generate the response
    # You might want to experiment with different generation parameters
    outputs = model.generate(
        input_ids=inputs["input_ids"], 
        attention_mask=inputs["attention_mask"],
        max_new_tokens=64,  # Max length for the generated answer
        num_beams=4,        # Use beam search for better quality
        early_stopping=True # Stop when all beam hypotheses have finished
    )
    
    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 3. Ask questions to the model
print("---")
print("Let's ask some questions to your fine-tuned model!")
print("---")

user_question_1 = "How can I return my shoes?"


---
Let's ask some questions to your fine-tuned model!
---


In [8]:
get_model_response(user_question_1)

'You can return your shoes to your home or business by contacting our customer service team or by contacting us.'

In [17]:
question = "what is email "

In [18]:
get_model_response(question)

"Email is a form of e-mail, and it's a form of e-mail."